In [0]:
# Alumno : Daniel Bolaños Martínez 76592621E
# Asignatura : Visión por Computador
# Práctica 2 : Introducción a Keras para la clasificación de imágenes

# -*- coding: utf-8 -*-

! pip install -q keras

In [0]:
#########################################################################
############ CARGAR LAS LIBRERÍAS NECESARIAS ############################
#########################################################################

# Importar librerías necesarias
import numpy as np
import keras
import matplotlib.pyplot as plt
import keras.utils as np_utils

# Importar modelos y capas que se van a usar
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
# Import Early Stopping
from keras.callbacks import EarlyStopping

# Importar el optimizador a usar
from keras.optimizers import SGD
# Importar el conjunto de datos
from keras.datasets import cifar100
# Importar DataGenerator
from keras.preprocessing.image import ImageDataGenerator

####Dimensiones constantes

In [0]:
# input image dimensions
img_rows, img_cols = 32, 32
num_classes = 25
batch_size = 32
epochs = 20

In [0]:
#########################################################################
######## FUNCIÓN PARA CARGAR Y MODIFICAR EL CONJUNTO DE DATOS ###########
#########################################################################

# A esta función sólo se le llama una vez. Devulve 4
# vectores conteniendo, por este orden, las imágenes
# de entrenamiento, las clases de las imágenes de
# entrenamiento, las im-agenes del conjunto de test y
# las clases del conjunto de test.

def cargarImagenes():
  # Cargamos Cifar100. Cada imagen tiene tamaño
  # (32, 32, 3). Nos vamos a quedar con las
  # imágenes de 25 de las clases.
  (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  
  x_train /= 255
  x_test /= 255
  
  train_idx = np.isin(y_train, np.arange(25))
  train_idx = np.reshape(train_idx, -1)
  x_train = x_train[train_idx]
  y_train = y_train[train_idx]
  
  test_idx = np.isin(y_test, np.arange(25))
  test_idx = np.reshape(test_idx, -1)
  x_test = x_test[test_idx]
  y_test = y_test[test_idx]

  # Transformamos los vectores de clases en matrices.
  # Cada componente se convierte en un vector de ceros
  # con un uno en la componente correspondiente a la
  # clase a la que pertenece la imagen. Este paso es
  # necesario para la clasificación multiclase en keras.
  
  y_train = np_utils.to_categorical(y_train, 25)
  y_test = np_utils.to_categorical(y_test, 25) 

  return x_train, y_train, x_test, y_test

#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

# Esta función devuelve el accuracy de un modelo, defi-
# nido como el porcentaje de etiquetas bien predichas
# frente al total de etiquetas. Como parámetros es
# necesario pasarle el vector de etiquetas verdaderas
# y el vector de etiquetas predichas, en el formato de
# keras (matrices donde cada etiqueta ocupa una fila,
# con un 1 en la posición de la clase a la que pertenece
# 0 en las demás).

def calcularAccuracy(labels, preds):
  labels = np.argmax(labels, axis=1)
  preds = np.argmax(preds, axis=1)
  
  accuracy = sum(labels == preds)/len(labels)
  
  return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################

# Esta función pinta dos gráficas, una con la evolución
# de la función de pérdida en el conjunto de train y
# en el de validación, y otra con la evolución del
# accuracy en el conjunto de train y el de validación.
# Es necesario pasarle como parámetro el historial del
# entrenamiento del modelo (lo que devuelven las 
# funciones fit() y fit_generator()))).

def mostrarEvolucion(hist):
  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()

  acc = hist.history['acc']
  val_acc = hist.history['val_acc']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy', 'Validation accuracy'])
  plt.show()

###Apartado 1: BaseNet en CIFAR100


In [0]:
#########################################################################
################## DEFINICIÓN DEL MODELO BASENET ########################
#########################################################################

x_train, y_train, x_test, y_test = cargarImagenes()

input_shape = (img_rows, img_cols, 3)

model = Sequential()
model.add(Conv2D(6, kernel_size=(5,5), activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [0]:
#########################################################################
######### DEFINICIÓN DEL OPTIMIZADOR Y COMPILACIÓN DEL MODELO ###########
#########################################################################

# explicar por qué selecciono ese optimizador

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['acc'])

# Una vez tenemos el modelo base, y antes de entrenar, vamos a guardar los
# pesos aleatorios con los que empieza la red, para poder reestablecerlos
# después y comparar resultados entre no usar mejoras y sí usarlas.

weights = model.get_weights()

In [0]:
#########################################################################
###################### ENTRENAMIENTO DEL MODELO #########################
#########################################################################

datagen_train = ImageDataGenerator(validation_split = 0.1)
train = datagen_train.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation = datagen_train.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = model.fit_generator(train, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation, validation_steps = len(x_train)*0.1/batch_size)

mostrarEvolucion(histograma)

In [0]:
#########################################################################
################ PREDICCIÓN SOBRE EL CONJUNTO DE TEST ###################
#########################################################################

preds = model.predict(x_test)
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = model.predict(x_train)
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))

###Apartado 2: Mejora del modelo

####Normalización

In [0]:
#########################################################################
########################## MEJORA DEL MODELO ############################
########################### NORMALIZACIÓN ###############################
#########################################################################

model.set_weights(weights)

datagen_train_norm = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True, validation_split = 0.1)
datagen_test_norm = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True)
datagen_train_norm.fit(x_train)
datagen_test_norm.fit(x_train)
train_norm = datagen_train_norm.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation_norm = datagen_train_norm.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = model.fit_generator(train_norm, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation_norm, validation_steps = len(x_train)*0.1/batch_size)

mostrarEvolucion(histograma)

preds = model.predict_generator(datagen_test_norm.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = model.predict_generator(datagen_train_norm.flow(x_train, batch_size = 1, shuffle = False), steps = len(x_train))
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))

####Aumento de datos

In [0]:
#########################################################################
########################## MEJORA DEL MODELO ############################
########################## AUMENTO DE DATOS #############################
#########################################################################

# Probar diversos parámetros para el aumento, sin flip, cambiar zoom_range...
# Justificar por qué puedo hacer flip (no cambio de la distribución en nuestro caso)

model.set_weights(weights)

datagen_train_zoom = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True,
                                        horizontal_flip=False, zoom_range=0.5, validation_split = 0.1)
datagen_test_zoom = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True)

datagen_train_zoom.fit(x_train)
datagen_test_zoom.fit(x_train)

train_zoom = datagen_train_zoom.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation_zoom = datagen_train_zoom.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = model.fit_generator(train_zoom, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation_zoom, validation_steps = len(x_train)*0.1/batch_size)

mostrarEvolucion(histograma)

preds = model.predict_generator(datagen_test_zoom.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = model.predict_generator(datagen_train_zoom.flow(x_train, batch_size = 1, shuffle = False), steps = len(x_train))
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))

####Red más profunda

In [0]:
# Crearemos un nuevo modelo añadiendo las capas que veamos necesarias

x_train, y_train, x_test, y_test = cargarImagenes()

input_shape = (img_rows, img_cols, 3)

my_model = Sequential()
my_model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape=input_shape))
my_model.add(Dropout(0.25))
my_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
my_model.add(MaxPooling2D(pool_size=(2,2)))
my_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
my_model.add(Dropout(0.25))
my_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
my_model.add(MaxPooling2D(pool_size=(2,2)))
my_model.add(Flatten())
my_model.add(Dense(512, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(256, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(num_classes, activation='softmax'))

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

my_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['acc'])

weights = my_model.get_weights()

# Aplicamos la normalización y un aumento de datos usando los parámetros que mejor resultado han obtenido

datagen_train_depth = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True, validation_split = 0.1)
datagen_test_depth = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True)

datagen_train_depth.fit(x_train)
datagen_test_depth.fit(x_train)

train_depth = datagen_train_depth.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation_depth = datagen_train_depth.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = my_model.fit_generator(train_depth, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation_depth, validation_steps = len(x_train)*0.1/batch_size)

mostrarEvolucion(histograma)

preds = my_model.predict_generator(datagen_test_depth.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = my_model.predict_generator(datagen_train_depth.flow(x_train, batch_size = 1, shuffle = False), steps = len(x_train))
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))

####Capas de normalización

In [0]:
# Crearemos un nuevo modelo añadiendo las capas que veamos necesarias

x_train, y_train, x_test, y_test = cargarImagenes()

input_shape = (img_rows, img_cols, 3)

# Probar a insertar BatchNormalization antes y después de las capas Dense

my_model2 = Sequential()
my_model2.add(Conv2D(32, kernel_size=(3,3), input_shape=input_shape))
my_model2.add(BatchNormalization())
my_model2.add(Activation('relu'))
my_model2.add(Dropout(0.25))
my_model2.add(Conv2D(64, kernel_size=(3,3)))
my_model2.add(BatchNormalization())
my_model2.add(Activation('relu'))
my_model2.add(MaxPooling2D(pool_size=(2,2)))
my_model2.add(Conv2D(64, kernel_size=(3,3)))
my_model2.add(BatchNormalization())
my_model2.add(Activation('relu'))
my_model2.add(Dropout(0.25))
my_model2.add(Conv2D(128, kernel_size=(3,3)))
my_model2.add(BatchNormalization())
my_model2.add(Activation('relu'))
my_model2.add(MaxPooling2D(pool_size=(2,2)))
my_model2.add(Flatten())
my_model2.add(Dense(512))
my_model2.add(BatchNormalization())
my_model2.add(Activation('relu'))
my_model2.add(Dropout(0.5))
my_model2.add(Dense(256))
my_model2.add(BatchNormalization())
my_model2.add(Activation('relu'))
my_model2.add(Dropout(0.5))
my_model2.add(Dense(num_classes, activation='softmax'))

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

my_model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['acc'])

weights = my_model2.get_weights()

# Aplicamos la normalización y un aumento de datos usando los parámetros que mejor resultado han obtenido

datagen_train_final = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True, validation_split = 0.1)
datagen_test_final = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True)

datagen_train_final.fit(x_train)
datagen_test_final.fit(x_train)

train_final = datagen_train_final.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation_final = datagen_train_final.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = my_model2.fit_generator(train_final, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation_final, validation_steps = len(x_train)*0.1/batch_size)

mostrarEvolucion(histograma)

preds = my_model2.predict_generator(datagen_test_final.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = my_model2.predict_generator(datagen_train_final.flow(x_train, batch_size = 1, shuffle = False), steps = len(x_train))
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))

####Early Stopping

In [0]:
# paramos cuando la gráfica de pérdida de validation comience a crecer
# y el valor accuracy de la validation se estabilice o decrezca

# probamos con 40 y buscamos el punto de parada

epochs = 40

my_model2.set_weights(weights)

datagen_train_final = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True, 
                                         horizontal_flip=True, zoom_range=0.2, validation_split = 0.1)
datagen_test_final = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True)

datagen_train_final.fit(x_train)
datagen_test_final.fit(x_train)

train_final = datagen_train_final.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation_final = datagen_train_final.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = my_model2.fit_generator(train_final, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation_final, validation_steps = len(x_train)*0.1/batch_size,
                    callbacks = [EarlyStopping(monitor = 'val_acc', patience = 4, restore_best_weights = True)])

mostrarEvolucion(histograma)

preds = my_model2.predict_generator(datagen_test_final.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = my_model2.predict_generator(datagen_train_final.flow(x_train, batch_size = 1, shuffle = False), steps = len(x_train))
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))